Instead of using `python-constraint`, we want to use Googles OR-Tools to solve this problem.
This is because it features [better performance and scalability](https://github.com/google/or-tools/blob/stable/ortools/sat/docs/troubleshooting.md#improving-performance-with-multiple-workers) for larger problems (mutli threaded optimizations).

In [28]:
from csv import DictReader


def load_data_from_csv(filepath):
    """Load data from CSV file"""
    courses = {}

    with open(filepath, 'r', encoding='utf-8') as file:
        reader = DictReader(file, delimiter=';')

        for row in reader:
            course = row['Studiengang']
            courses[course] = {
                'num_groups': int(row['Projektgruppen']),
                'num_committees': int(row['Kommissionen']),
                'groups': [f'{course[:3]}-G{i + 1}' for i in range(int(row['Projektgruppen']))],
                'committees': [f'{course[:3]}-C{i + 1}' for i in range(int(row['Kommissionen']))],
            }

    return courses

Define all static variables from the assignment description
These should be immutable, but since there is no easy way to enforce this in Python we add the Final "type" so IDE warns us when trying to alter the variables



In [29]:
from typing import Final

# Constraint Values
num_presentations_per_group: Final = 3
num_max_commitee_room_changes_per_day: Final = 2

# Mappings
all_rooms: Final = ["L1", "L2", "L3"]
all_time_slots: Final = ["8-9", "10-11", "13-14", "15-16"]
all_days: Final = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
all_presentations: Final = range(num_presentations_per_group)

# Counts
num_rooms: Final = len(all_rooms)
num_slots: Final = len(all_time_slots)
num_days: Final = len(all_days)
num_max_presentations: Final = num_days * num_slots * num_rooms

# Will be filled from dataset
all_courses = {}
num_courses = 0
num_groups = {}
num_committees = {}

# Load dataset from CSV and process
data = load_data_from_csv("DS_CSP_1/pr_conf_001.csv")
num_courses = len(data)
all_courses = list(data.keys())
for key, course in data.items():
    num_groups[key] = course['num_groups']
    num_committees[key] = course['num_committees']


#### Creating variables

In [30]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()
presentations = {}
committee_assigned = {}

# This array defines assignments for presentations to groups of a course.
# presentations[(c, g, d, t, r)] equals 1 if room r is assigned to slot t on day d for group g in course c
for c in all_courses:
    for g in range(num_groups[c]):
        for d in all_days:
            for t in all_time_slots:
                for r in all_rooms:
                    presentations[(c, g, d, t, r)] = model.NewBoolVar(f'p({c},{g},{d},{t},{r})')

# This array defines
for c in all_courses:
    for cm in range(num_committees[c]):
        for d in all_days:
            for t in all_time_slots:
                for r in all_rooms:
                    committee_assigned[(c, cm, d, t, r)] = model.NewBoolVar(f'cm({c},{cm},{d},{t},{r})')

print(f'Created {len(presentations)} presentation assignments.')
print(f'Created {len(committee_assigned)} committee assignments.')

Created 1200 presentation assignments.
Created 660 committee assignments.


#### Adding constraints

In [31]:
# Each course can only have one presentation a day
for c in all_courses:
    for g in range(num_groups[c]):
        for d in all_days:
            model.add_at_most_one(
                presentations[(c, g, d, t, r)]
                for t in all_time_slots
                for r in all_rooms
            )

# Each group needs to present exactly num_presentations_per_group times (3).
for c in all_courses:
    for g in range(num_groups[c]):
        presentations_given = []
        for d in all_days:
            for t in all_time_slots:
                for r in all_rooms:
                    presentations_given.append(presentations[(c, g, d, t, r)])
        model.add(sum(presentations_given) == num_presentations_per_group)

# Make sure each room can host only 1 presentation per time slot
for d in all_days:
    for t in all_time_slots:
        for r in all_rooms:
            model.add_at_most_one(
                presentations[(c, g, d, t, r)]
                for c in all_courses
                for g in range(num_groups[c])
            )

# Each committee can only be assigned to one room at a time
for c in all_courses:
    for cm in range(num_committees[c]):
        for d in all_days:
            for t in all_time_slots:
                model.AddAtMostOne(
                    committee_assigned[(c, cm, d, t, r)]
                    for r in all_rooms
                )

# Each presentation needs exactly 1 committee from the right course to be present
for c in all_courses:
    for d in all_days:
        for t in all_time_slots:
            for r in all_rooms:
                # If any group from course c presents at (d,t,r)
                groups_presenting = sum(presentations[(c, g, d, t, r)] for g in range(num_groups[c]))

                # Exactly one committee instance from course c must be there
                committees_present = sum(
                    committee_assigned[(c, cm, d, t, r)]
                    for cm in range(num_committees[c])
                )

                model.Add(groups_presenting == committees_present)


In [32]:
# Calculate if problem is feasible
total_presentations_needed = sum(num_groups[c] * num_presentations_per_group for c in all_courses)
total_slots_available = num_days * num_slots * num_rooms

print(f"Total presentations needed: {total_presentations_needed}")
print(f"Total slots available: {total_slots_available}")

solver = cp_model.CpSolver()
status = solver.solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solution found!")
    for d in all_days:
        print(f"\nDay {d}:")
        for t in all_time_slots:
            print(f"  Time {t}:")
            for r in all_rooms:
                print(f"    Room {r}:", end=" ")
                found = False
                for c in all_courses:
                    for g in range(num_groups[c]):
                        if solver.value(presentations[(c, g, d, t, r)]) == 1:
                            # Find which committee is assigned
                            committee_idx = None
                            for cm in range(num_committees[c]):
                                if solver.value(committee_assigned[(c, cm, d, t, r)]) == 1:
                                    committee_idx = cm + 1
                                    break
                            print(f"{c}, G{g}, C{committee_idx}", end="")
                            found = True
                if not found:
                    print("Empty", end="")
                print()
else:
    print("No solution found!")


Total presentations needed: 60
Total slots available: 60
Solution found!

Day Monday:
  Time 8-9:
    Room L1: Elektrotechnik, G1, C1
    Room L2: Maschinenbau, G2, C2
    Room L3: Informatik, G2, C3
  Time 10-11:
    Room L1: Maschinenbau, G1, C2
    Room L2: Informatik, G5, C1
    Room L3: Maschinenbau, G3, C1
  Time 13-14:
    Room L1: Maschinenbau, G0, C1
    Room L2: Informatik, G6, C2
    Room L3: Informatik, G1, C3
  Time 15-16:
    Room L1: Elektrotechnik, G0, C2
    Room L2: Elektrotechnik, G3, C1
    Room L3: Data Science, G1, C1

Day Tuesday:
  Time 8-9:
    Room L1: Informatik, G7, C3
    Room L2: Elektrotechnik, G2, C1
    Room L3: Informatik, G4, C1
  Time 10-11:
    Room L1: Data Science, G1, C1
    Room L2: Informatik, G1, C3
    Room L3: Informatik, G3, C2
  Time 13-14:
    Room L1: Informatik, G5, C3
    Room L2: Elektrotechnik, G0, C2
    Room L3: Elektrotechnik, G3, C1
  Time 15-16:
    Room L1: Maschinenbau, G3, C1
    Room L2: Data Science, G0, C1
    Room L3: Inf